In [65]:
#Importing dependencies and libraries
import numpy as np
import json
import requests
import pandas as pd

from config import api_key
from citipy import citipy

In [68]:
#Building the 500+ cities list

#setting ranges to look for cities
latitude_range = np.arange(-90,91,5)
longitude_range = np.arange(-180,181,5)

#nested for loop that will retrieve every city
cities_list = []

for x in longitude_range:
    for y in latitude_range:
        city = citipy.nearest_city(x,y)
        cities_list.append(city.city_name)

#converting the resulting list to a dataframe, dropping the duplicates and reindexing
cities_list_df = pd.DataFrame({'Cities':cities_list})
cities_list_df = cities_list_df.drop_duplicates().reset_index(drop = True)
cities_list_df

,Cities
0,ushuaia
1,bredasdorp
2,kruisfontein
3,port elizabeth
4,port alfred
...,...
604,skalistyy
605,dikson
606,longyearbyen
607,vardo


In [74]:
#API requests

url = "http://api.openweathermap.org/data/2.5/weather?"
city = "London"

# Build query URL
query_url = url + "appid=" + api_key + "&q=" + city + "&units=metric" # Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()

# Get the temperature from the response
print(f"The weather API responded with: {weather_json}.")

The weather API responded with: {'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 12.77, 'feels_like': 11.33, 'temp_min': 11.11, 'temp_max': 15, 'pressure': 1028, 'humidity': 47}, 'visibility': 10000, 'wind': {'speed': 6.69, 'deg': 90}, 'clouds': {'all': 6}, 'dt': 1619108009, 'sys': {'type': 1, 'id': 1414, 'country': 'GB', 'sunrise': 1619066978, 'sunset': 1619118478}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}.


In [78]:
# Pulling relevant data from the response

# Northern Hemisphere - Wind Speed (mph) vs. Latitude
# Southern Hemisphere - Wind Speed (mph) vs. Latitude

latitude = weather_json['coord']['lat']
longitude = weather_json['coord']['lon']
temperature = weather_json['main']['temp']
humidity = weather_json['main']['humidity']
cloudiness = weather_json['clouds']['all']
wind = weather_json['wind']['speed']
print (latitude)
print (longitude)
print (temperature)
print (humidity)
print (cloudiness)
print (wind)

51.5085
-0.1257
12.77
47
6
6.69
